# LR & DA

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from main import CIFAR10_dataset, CIFAR10_trainer
import torch
import torchvision
import torch.nn as nn
from tqdm import tqdm
import multiprocessing
import torch.optim as optim
import torch.nn.functional as  F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import v2
from torch.utils.data import default_collate

In [3]:
def add_gaussian_noise(img, mean=0, std=0.1):
    noise = torch.randn(img.size()) * std + mean
    return img + noise

# Transformaciones para entrenamiento
train_transforms = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(degrees=15),
    transforms.RandomGrayscale(p=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    
    transforms.ToTensor(),
    # transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3)),
    transforms.Lambda(lambda x: add_gaussian_noise(x, 0, 0.1)),
])
train_dataset = CIFAR10_dataset(partition="train", transform=train_transforms)
test_dataset = CIFAR10_dataset(partition="test")

# CutMix and MixUp
cutmix = v2.CutMix(num_classes=10)
mixup = v2.MixUp(num_classes=10)
cutmix_or_mixup = v2.RandomChoice([cutmix, mixup])

def collate_fn(batch):
    data = default_collate(batch)  # Asegura el formato (inputs, targets)
    inputs, labels= cutmix_or_mixup(data['img'], data['label']) # Aplica CutMix o MixUp
    return {"img": inputs, "label": labels}

####################################################################
# DataLoader Class
####################################################################

batch_size = 150
num_workers = multiprocessing.cpu_count()-1
print("Num workers", num_workers)
train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=num_workers, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=num_workers)


Loading CIFAR10  train  Dataset...
Files already downloaded and verified
	Total Len.:  50000 
 --------------------------------------------------

Loading CIFAR10  test  Dataset...
Files already downloaded and verified
	Total Len.:  10000 
 --------------------------------------------------
Num workers 11


In [12]:
class BasicBlock(nn.Module):
    expansion = 1  # Para ResNet18/34, el factor de expansión es 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None, dropout=0.5):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample  # Para ajustar dimensiones si es necesario
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        identity = x  # Shortcut

        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.dropout(out)

        out += identity  # Residual connection
        out = self.relu(out)
        

        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super(ResNet, self).__init__()
        self.in_channels = 32

        # Inicial: Convolución, BatchNorm y ReLU
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)

        # Bloques residuales

        self.layer1 = self._make_layer(block, 32, layers[0], dropout=0.05)
        self.layer2 = self._make_layer(block, 64, layers[1], stride=2, dropout=0.1)
        self.layer3 = self._make_layer(block, 128, layers[2], stride=2, dropout=0.2)
        self.layer4 = self._make_layer(block, 256, layers[3], stride=2, dropout=0.3)
 

        # Clasificación
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256 * block.expansion, 512)
        self.fc2 = nn.Linear(512,num_classes)
        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.5)

    def _make_layer(self, block, out_channels, blocks, stride=1, dropout=0.5):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion)
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample, dropout))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout1(x)
        x = self.fc(x)
        x = self.dropout2(x)
        x = self.fc2(x)

        return x
    
def ResNet18(num_classes=1000):
    return ResNet(BasicBlock, [3, 2, 2, 2], num_classes=num_classes)

In [13]:
# Instantiating the network and printing its architecture
num_classes = 10
net = ResNet18(num_classes)
print(net)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Params: ", count_parameters(net))

####################################################################
# Training settings
####################################################################

# Training hyperparameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, weight_decay=1e-6, momentum=0.9)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=10, min_lr=0.00001)
epochs = 100

ResNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (dropout): Dropout(p=0.05, inplace=False)
    )
    (1): BasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1

In [14]:
trainer = CIFAR10_trainer(net, train_dataloader, test_dataloader, optimizer,criterion, epochs, lr_scheduler, batch_size=batch_size)

In [15]:
trainer.train()


---- Start Training ----


Test 0: 100%|██████████| 67/67 [00:02<00:00, 32.96batch/s]


[Epoch 1] Train Loss: 2.111084 - Test Loss: 1.588423 - Train Error: 75.65% - Test Error: 58.40%


Test 1: 100%|██████████| 67/67 [00:02<00:00, 29.97batch/s]

[Epoch 2] Train Loss: 1.890589 - Test Loss: 1.790028 - Train Error: 62.49% - Test Error: 60.37%


...

[Epoch 98] Train Loss: 1.110442 - Test Loss: 0.619774 - Train Error: 21.49% - Test Error: 13.66%



Test 98: 100%|██████████| 67/67 [00:02<00:00, 29.88batch/s]

[Epoch 99] Train Loss: 1.126361 - Test Loss: 0.550979 - Train Error: 23.94% - Test Error: 13.59%



Test 99: 100%|██████████| 67/67 [00:02<00:00, 29.87batch/s]

[Epoch 100] Train Loss: 1.063689 - Test Loss: 0.577048 - Train Error: 20.85% - Test Error: 13.90%

BEST TEST ERROR:  13.54  in epoch  90
